In [1]:
# Load packages
from datetime import datetime
import logging
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import scipy
import sys

In [2]:
# declare universal variables
CONDITION = 'severePE'
CONDITION_CLINICAL_DF = 'Severe PE'
SUFFIX = ' (protein)'

In [3]:
# Set up logger
os.remove('spearman.standardized.interomics.' + CONDITION + '.pairs.all.miniDP3.log')
logger = logging.getLogger('analytics')

# Set the logging level
logger.setLevel(logging.INFO)

# Define a log format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Create a console handler to output log messages to the console
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# (Optional) Create a file handler to log messages to a file
file_handler = logging.FileHandler('spearman.standardized.interomics.' + CONDITION + '.pairs.all.miniDP3.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [4]:
dfpb = pd.read_table('../data/proteins-final-adjusted.csv', sep=',', dtype={'Patient-ID':str})
dfmb = pd.read_table('../data/metabolites-final-adjusted.csv', sep=',', dtype={'Patient-ID':str})
dfmirna = pd.read_table('../data/mirna-final-adjusted.csv', sep=',', dtype={'Patient-ID':str})
dftb = pd.read_table('../data/transcripts-final-adjusted.csv', sep=',', dtype={'Patient-ID':str})
dfph = pd.read_table('../data/PlacentalHistopathologyReportsCleaned.csv', sep=',', dtype={'Patient-ID':str})
dfph2 = pd.read_table('../data/PlacentalHistopathologySlideFeaturesCleaned.csv', sep=',', dtype={'Patient-ID':str})

dfp_analyte = pd.read_table('../data/protein-meta-final.csv', sep=',')
dfm_analyte = pd.read_table('../data/metabolites-meta-final.csv', sep=',')
dfmirna_analyte = pd.read_table('../data/mirna-meta-final.csv', sep=',')
dft_analyte = pd.read_table('../data/transcripts-meta-final.csv', sep=',')

df_clinical = pd.read_table('../data/metadata-final.csv', sep=',', dtype={'Study_ID/ID':str})
df_clinical = df_clinical[df_clinical['Condition']==CONDITION_CLINICAL_DF]

In [5]:
dfpb.shape, dfmb.shape, dfmirna.shape, dftb.shape, dfph.shape, dfph2.shape, df_clinical.shape

((321, 352), (319, 866), (294, 450), (294, 9585), (275, 6), (348, 9), (75, 21))

# Set up annotations

In [6]:
metabolites = {}
for index, row in dfm_analyte.iterrows():
    metabolites[row['CHEMICAL_NAME']] = {'name': row['CHEMICAL_NAME'], 'sub_group': row['SUB_PATHWAY'], 'super_group': row['SUPER_PATHWAY']}
    
miRNAs = {}
for index, row in dfmirna_analyte.iterrows():
    miRNAs[row['Transcript']] = {'name': row['Transcript'], 'sub_group': None, 'super_group': None}
    
proteins = {}
for index, row in dfp_analyte.iterrows():
    proteins[row['Assay'] + SUFFIX] = {'name': row['Assay'], 'sub_group': None, 'super_group': row['Panel']}
    
transcripts = {}
for index, row in dft_analyte.iterrows():
    transcripts[row['Transcript']] = {'name': row['Transcript'], 'sub_group': None, 'super_group': None}
    
histopathology = {}
for item in dfph:
    if item != 'Patient-ID':
        histopathology[item] = {'name': item, 'sub_group': None, 'super_group': None}
for item in dfph2:
    if item != 'Patient-ID':
        histopathology[item] = {'name': item, 'sub_group': None, 'super_group': None}
        
# print the number of included analytes
print('Histopathology Features:', str(len(histopathology)))
print('Metabolites:', str(len(metabolites)))
print('miRNAs:', str(len(miRNAs)))
print('Proteins:', str(len(proteins)))
print('Transcripts:', str(len(transcripts)))
print('Total:', str(len(histopathology)+len(metabolites)+len(miRNAs)+len(proteins)+len(transcripts)))

Histopathology Features: 13
Metabolites: 1032
miRNAs: 2414
Proteins: 452
Transcripts: 51174
Total: 55085


# Merge against prot/met data

In [7]:
dfm_analyte = list(set(metabolites.keys()).intersection(dfmb.columns))
dfmirna_analyte = list(set(miRNAs.keys()).intersection(dfmirna.columns))
dfp_analyte = list(set(proteins.keys()).intersection(dfpb.columns))
dft_analyte = list(set(transcripts.keys()).intersection(dftb.columns))
dfph_measurement = ['MVM', 'FVM', 'AI', 'CI', 'Placental Abnormality']
dfph_measurement2 = ['DVH', 'AVM', 'Syncytial Knots', 'Segmental Avascular Villi', 'DVM', 'Villitis of Unknown Etiology', 'Diffuse Villous Edema', 'Chorangiosis']

# Get all analytes together
analytes = dfph_measurement + dfph_measurement2 + dfm_analyte + dfmirna_analyte + dfp_analyte + dft_analyte

# print the number of included analytes that pass missingness threshold
print('Histopathology Features:', str(len(dfph_measurement)+len(dfph_measurement2)))
print('Metabolites:', str(len(dfm_analyte)))
print('miRNAs:', str(len(dfmirna_analyte)))
print('Proteins:', str(len(dfp_analyte)))
print('Transcripts:', str(len(dft_analyte)))
print('Total:', str(len(analytes)))

Histopathology Features: 13
Metabolites: 865
miRNAs: 448
Proteins: 343
Transcripts: 9582
Total: 11251


In [8]:
df_clinical.head()

,Patient-ID,Condition,LaborInitiation,MaternalAge,isWhite,isBlack,isAsian,Ethnicity,WksGest,PregravidBMI,...,Para,LaborOnset,Smoker,IllicitDrugUser,DeliveryMethod,PregnancyRelatedHypertension,FetalGrowthRestriction,Preeclampsia,InfSex,Birthweight
2,KH-1132,Severe PE,0,31.0,0.0,0.0,1.0,0.0,38.000000,26.297578,...,0.0,0.0,0.0,0.0,0.0,1,0,1,1.0,3490.0
3,KH-1149,Severe PE,0,29.0,0.0,0.0,0.0,0.0,33.857143,23.828125,...,1.0,1.0,0.0,1.0,0.0,1,0,1,0.0,1545.0
21,MJ-0128,Severe PE,0,19.0,0.0,1.0,0.0,0.0,36.000000,25.396825,...,0.0,1.0,1.0,0.0,0.0,1,0,1,1.0,2635.0
26,MJ-0296,Severe PE,0,36.0,1.0,0.0,0.0,0.0,34.000000,28.997474,...,1.0,0.0,0.0,0.0,1.0,1,0,1,0.0,1917.0
33,MJ-0363,Severe PE,0,19.0,0.0,1.0,0.0,0.0,38.857143,27.700831,...,0.0,0.0,0.0,0.0,1.0,1,0,1,1.0,3130.0


In [9]:
df = df_clinical.merge(dfph[['Patient-ID'] + list(dfph_measurement)], on=['Patient-ID'], how='left')
print(df.shape)
df = df.merge(dfph2[['Patient-ID'] + list(dfph_measurement2)], on=['Patient-ID'], how='left')
print(df.shape)
df = df.merge(dfpb[['Patient-ID'] + list(dfp_analyte)], on=['Patient-ID'], how='inner')
print(df.shape)
df = df.merge(dfmb[['Patient-ID'] + list(dfm_analyte)], on=['Patient-ID'], how='left')
print(df.shape)
df = df.merge(dfmirna[['Patient-ID'] + list(dfmirna_analyte)], on=['Patient-ID'], how='left')
print(df.shape)
df = df.merge(dftb[['Patient-ID'] + list(dft_analyte)], on=['Patient-ID'], how='left')
print(df.shape)
df = df.fillna(-1)

(75, 26)
(75, 34)
(71, 377)
(71, 1242)
(71, 1690)
(71, 11272)


In [10]:
df.shape, dfpb.shape, dfmb.shape, dfmirna.shape, dftb.shape, dfph.shape, dfph2.shape, df_clinical.shape

((71, 11272),
 (321, 352),
 (319, 866),
 (294, 450),
 (294, 9585),
 (275, 6),
 (348, 9),
 (75, 21))

In [11]:
df['InfSex'].value_counts()

InfSex
0.0    40
1.0    31
Name: count, dtype: int64

In [12]:
df['isWhite'].value_counts()

isWhite
 1.0    45
 0.0    25
-1.0     1
Name: count, dtype: int64

In [13]:
df['isBlack'].value_counts()

isBlack
 0.0    52
 1.0    18
-1.0     1
Name: count, dtype: int64

In [14]:
df['isAsian'].value_counts()

isAsian
 0.0    65
 1.0     5
-1.0     1
Name: count, dtype: int64

In [15]:
df.head()

,Patient-ID,Condition,LaborInitiation,MaternalAge,isWhite,isBlack,isAsian,Ethnicity,WksGest,PregravidBMI,...,CRBN,JADE3,ABT1,ACO2,SPEF2,SEPTIN7,ZNF395,REXO1,IP6K1,MAGED1
0,KH-1132,Severe PE,0,31.0,0.0,0.0,1.0,0.0,38.000000,26.297578,...,1060.0,1082.0,497.0,1640.0,234.0,1610.0,680.0,1480.0,1627.0,1693.0
1,KH-1149,Severe PE,0,29.0,0.0,0.0,0.0,0.0,33.857143,23.828125,...,944.0,1275.0,348.0,1633.0,200.0,1624.0,962.0,1597.0,1089.0,1723.0
2,MJ-0128,Severe PE,0,19.0,0.0,1.0,0.0,0.0,36.000000,25.396825,...,679.0,839.0,275.0,1303.0,105.0,1159.0,394.0,1328.0,912.0,1256.0
3,MJ-0296,Severe PE,0,36.0,1.0,0.0,0.0,0.0,34.000000,28.997474,...,814.0,1181.0,277.0,1089.0,167.0,1521.0,606.0,1157.0,1286.0,933.0
4,MJ-0363,Severe PE,0,19.0,0.0,1.0,0.0,0.0,38.857143,27.700831,...,718.0,745.0,264.0,1241.0,154.0,1244.0,500.0,1717.0,1297.0,1522.0


# Correlation network

In [16]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.sandbox.stats.multicomp
from statsmodels.genmod.families import family, links
import itertools

In [17]:
drop_cols = ['Patient-ID', 'Grav', 'DeliveryMethod', 
             'Para', 'PrePregWt_Kg', 'Ethnicity', 'isAsian', 
             'isBlack', 'WksGest', 'Condition', 'isWhite', 
             'Race', 'MaternalAge', 'Smoke', 'PregravidBMI',
             'HeightMeters']

# Pairwise GLM

In [18]:
def count_pairs(dat, analytes, proteins, metabolites, mirnas, transcripts, histopathology):

    # Create all possible pairs of inputs
    pairs = list(itertools.combinations(analytes, 2))
    logger.info('Created {} pairs'.format(len(pairs)))
         
    count = 0 
    done = 0
    skipped = 0
    completed = 0

    results = []
    columns = list(dat.columns)

    start_time = datetime.now()
    for (col1, col2) in pairs:

        if (col1 == col2):
            continue
                    
        if (col1 in histopathology.keys()) and (col2 in histopathology.keys()):
            continue
        
        if (col1 in metabolites.keys()) and (col2 in metabolites.keys()):
            continue
            
        if (col1 in mirnas.keys()) and (col2 in mirnas.keys()):
            continue

        if (col1 in proteins.keys()) and (col2 in proteins.keys()):
            continue
        
        if (col1 in transcripts.keys()) and (col2 in transcripts.keys()):
            continue
            
        if (col1 in proteins.keys()):
            name1 = proteins[col1]['name']
            subgroup1 = proteins[col1]['sub_group'] 
            supergroup1 = proteins[col1]['super_group']
            type1 = 'protein'
        elif (col1 in metabolites.keys()):
            name1 = metabolites[col1]['name']
            subgroup1 = metabolites[col1]['sub_group'] 
            supergroup1 = metabolites[col1]['super_group']
            type1 = 'metabolite'
        elif (col1 in mirnas.keys()):
            name1 = mirnas[col1]['name']
            subgroup1 = mirnas[col1]['sub_group'] 
            supergroup1 = mirnas[col1]['super_group']
            type1 = 'miRNA'
        elif (col1 in transcripts.keys()):
            name1 = transcripts[col1]['name']
            subgroup1 = transcripts[col1]['sub_group'] 
            supergroup1 = transcripts[col1]['super_group']
            type1 = 'transcript'
        elif (col1 in histopathology.keys()):
            name1 = histopathology[col1]['name']
            subgroup1 = histopathology[col1]['sub_group'] 
            supergroup1 = histopathology[col1]['super_group']
            type1 = 'histopathology'

        else:
            logger.warning('Col1 {} not found in any annotation'.format(col1))
            name1 = None
            subgroup1 = None
            supergroup1 = None
            type1 = None
            
        if (col2 in proteins.keys()):
            name2 = proteins[col2]['name']
            subgroup2 = proteins[col2]['sub_group'] 
            supergroup2 = proteins[col2]['super_group']
            type2 = 'protein'
        elif (col2 in metabolites.keys()):
            name2 = metabolites[col2]['name']
            subgroup2 = metabolites[col2]['sub_group'] 
            supergroup2 = metabolites[col2]['super_group']
            type2 = 'metabolite'
        elif (col2 in mirnas.keys()):
            name2 = mirnas[col2]['name']
            subgroup2 = mirnas[col2]['sub_group'] 
            supergroup2 = mirnas[col2]['super_group']
            type2 = 'miRNA'
        elif (col2 in transcripts.keys()):
            name2 = transcripts[col2]['name']
            subgroup2 = transcripts[col2]['sub_group'] 
            supergroup2 = transcripts[col2]['super_group']
            type2 = 'transcript'
        elif (col2 in histopathology.keys()):
            name2 = histopathology[col2]['name']
            subgroup2 = histopathology[col2]['sub_group'] 
            supergroup2 = histopathology[col2]['super_group']
            type2 = 'histopathology'
    
        else:
            logger.warning('Col2 {} not found in any annotation'.format(col2))
            name2 = None
            subgroup2 = None
            supergroup2 = None
            type2 = None
            
        count += 1
        
    print('Running {} pairs'.format(count))

count_pairs(df, analytes, proteins, metabolites, miRNAs, transcripts, histopathology)

2024-02-04 20:55:44,607 - analytics - INFO - Created 63286875 pairs


Running 16851765 pairs


In [19]:
def evaluate_effect_size(df, logger):
    # evaluate effect size - cohen's d
    df_no_effect = df[abs(df['coef'])<0.01]
    df_very_small = df[abs(df['coef'])>=0.01]
    df_very_small = df_very_small[abs(df_very_small['coef'])<0.1]
    df_small = df[abs(df['coef'])>=0.1]
    df_small = df_small[abs(df_small['coef'])<0.2]
    df_medium = df[abs(df['coef'])>=0.2]
    df_medium  = df_medium[abs(df_medium['coef'])<0.3]
    df_medium_large = df[abs(df['coef'])>=0.3]
    df_medium_large = df_medium_large[abs(df_medium_large['coef'])<0.5]
    df_large = df[abs(df['coef'])>=0.5]
    df_large = df_large[abs(df_large['coef'])<0.7]
    df_very_large = df[abs(df['coef'])>=0.7]
    df_very_large = df_very_large[abs(df_very_large['coef'])<0.9]
    df_huge = df[abs(df['coef'])>=0.9]
    
    # print info on effect sizes
    logger.info('Number of Analyte Pairs with No Monotonic Relationship: {}'.format(str(df_no_effect.shape[0])))
    logger.info('Number of Analyte Pairs with Very Weak Monotonic Relationship: {}'.format(str(df_very_small.shape[0])))
    logger.info('Number of Analyte Pairs with Weak Monotonic Relationship: {}'.format(str(df_small.shape[0])))
    logger.info('Number of Analyte Pairs with Moderate Monotonic Relationship: {}'.format(str(df_medium.shape[0])))
    logger.info('Number of Analyte Pairs with Moderate Strong Monotonic Relationship: {}'.format(str(df_medium_large.shape[0])))
    logger.info('Number of Analyte Pairs with Strong Monotonic Relationship: {}'.format(str(df_large.shape[0])))
    logger.info('Number of Analyte Pairs with Very Strong Monotonic Relationship: {}'.format(str(df_very_large.shape[0])))
    logger.info('Number of Analyte Pairs with Near Perfect Monotonic Relationship: {}'.format(str(df_huge.shape[0])))
    logger.info('\n')
    

def run_spearman_pairwise(dat, condition, analytes, proteins, metabolites, mirnas, transcripts, histopathology, max_run=None):

    # Create all possible pairs of inputs
    pairs = list(itertools.combinations(analytes, 2))
    logger.info('Created {} pairs'.format(len(pairs)))
    
    f_failed = open('failed.spearman.standardized.interomics.' + condition + '.pairs.all.miniDP3.txt', mode='w')
    f_output = 'spearman.standardized.interomics.' + condition + '.all.miniDP3.txt'
    f_output_sig = 'spearman.standardized.interomics.' + condition + '.sig.miniDP3.txt'
         
    count = 0 
    done = 0
    skipped = 0
    completed = 0

    results = []
    columns = list(dat.columns)

    start_time = datetime.now()
    for (col1, col2) in pairs:

        if (col1 == col2):
            continue
                           
        if (col1 in histopathology.keys()) and (col2 in histopathology.keys()):
            continue
        
        if (col1 in metabolites.keys()) and (col2 in metabolites.keys()):
            continue
            
        if (col1 in mirnas.keys()) and (col2 in mirnas.keys()):
            continue

        if (col1 in proteins.keys()) and (col2 in proteins.keys()):
            continue
        
        if (col1 in transcripts.keys()) and (col2 in transcripts.keys()):
            continue

        if (col1 in proteins.keys()):
            name1 = proteins[col1]['name']
            subgroup1 = proteins[col1]['sub_group'] 
            supergroup1 = proteins[col1]['super_group']
            type1 = 'protein'
        elif (col1 in metabolites.keys()):
            name1 = metabolites[col1]['name']
            subgroup1 = metabolites[col1]['sub_group'] 
            supergroup1 = metabolites[col1]['super_group']
            type1 = 'metabolite'
        elif (col1 in mirnas.keys()):
            name1 = mirnas[col1]['name']
            subgroup1 = mirnas[col1]['sub_group'] 
            supergroup1 = mirnas[col1]['super_group']
            type1 = 'miRNA'
        elif (col1 in transcripts.keys()):
            name1 = transcripts[col1]['name']
            subgroup1 = transcripts[col1]['sub_group'] 
            supergroup1 = transcripts[col1]['super_group']
            type1 = 'transcript'
        elif (col1 in histopathology.keys()):
            name1 = histopathology[col1]['name']
            subgroup1 = histopathology[col1]['sub_group'] 
            supergroup1 = histopathology[col1]['super_group']
            type1 = 'histopathology'
        else:
            logger.warning('Col1 {} not found in any annotation'.format(col1))
            name1 = None
            subgroup1 = None
            supergroup1 = None
            type1 = None
            
        if (col2 in proteins.keys()):
            name2 = proteins[col2]['name']
            subgroup2 = proteins[col2]['sub_group'] 
            supergroup2 = proteins[col2]['super_group']
            type2 = 'protein'
        elif (col2 in metabolites.keys()):
            name2 = metabolites[col2]['name']
            subgroup2 = metabolites[col2]['sub_group'] 
            supergroup2 = metabolites[col2]['super_group']
            type2 = 'metabolite'
        elif (col2 in mirnas.keys()):
            name2 = mirnas[col2]['name']
            subgroup2 = mirnas[col2]['sub_group'] 
            supergroup2 = mirnas[col2]['super_group']
            type2 = 'miRNA'
        elif (col2 in transcripts.keys()):
            name2 = transcripts[col2]['name']
            subgroup2 = transcripts[col2]['sub_group'] 
            supergroup2 = transcripts[col2]['super_group']
            type2 = 'transcript'
        elif (col2 in histopathology.keys()):
            name2 = histopathology[col2]['name']
            subgroup2 = histopathology[col2]['sub_group'] 
            supergroup2 = histopathology[col2]['super_group']
            type2 = 'histopathology'
        else:
            logger.warning('Col2 {} not found in any annotation'.format(col2))
            name2 = None
            subgroup2 = None
            supergroup2 = None
            type2 = None

        sub = dat[['Patient-ID', col1, col2]].copy()
        sub.rename(columns={col1:'analyte1'}, inplace=True)
        sub.rename(columns={col2:'analyte2'}, inplace=True)


        if (sub['analyte1'].skew() > 1.5) | (sub['analyte1'].skew() < -1.5):

            #logger.info('Setting gamma family for skewed analyte %s'%(col))

            # Set any zero values to 1/2 the smallest value
            sub.loc[sub['analyte1']==0, 'analyte1'] = (sub.loc[sub['analyte1']>0, 'analyte1'].min() / 2.0)
            

        # z-score transform dependent variable data
        if col2 not in histopathology.keys():
            sub['analyte2'] = sub['analyte2'].astype(np.float64)
            sub['analyte2'] = (sub['analyte2'] - sub['analyte2'].mean() ) / sub['analyte2'].std()

            
        try:
            coef, p = scipy.stats.spearmanr(sub['analyte1'], sub['analyte2'])
            results.append((col1, name1, type1, supergroup1, subgroup1, col2, name2, type2, supergroup2, subgroup2, coef, p))
            completed += 1

        except Exception as e:
            logger.info('Failed analytes {} {} with error {}'.format(col1, col2, str(e)))
            f_failed.write(col1 + '\t' + col2 + '\n')
            
            
        count += 1
        if (max_run is not None) and (count >= max_run):
            break
        
        if (count % 1000) == 0:
            elapsed_time = datetime.now() - start_time    
            logger.info('Finished {} in {:.3f} seconds (skipped {})'.format(count, elapsed_time.total_seconds(), skipped))
            
    df = pd.DataFrame(results, columns=['col1', 'name1', 'type1', 'supergroup1', 'subgroup1', 'col2', 'name2', 'type2', 'supergroup2', 'subgroup2', 'coef', 'p-value'])
    df.sort_values(['p-value'], ascending=True, inplace=True)
    f_failed.close()

    np.seterr(all='warn')
    logger.info('\nFinished analyzing all {} pairs!'.format(str(count)))
    logger.info('Number of analyte pairs with converged formula: {}'.format(str(df.shape[0])))
    (adj_pval_index, adj_pval, _, _) = statsmodels.sandbox.stats.multicomp.multipletests(df.loc[~df['p-value'].isnull(), 'p-value'], alpha=0.05, method='fdr_bh')
    df.loc[~df['p-value'].isnull(), 'pval_adj'] = adj_pval
    df.sort_values(['pval_adj'], ascending=True, inplace=True)
    df.to_csv(f_output, index=False, sep='\t')
    
    # filter for significance
    df = df[df['pval_adj']<0.05]  #benjammini-hochman
    #bonferroni_correction = 0.05/count
    #df = df[df['p-value']<bonferroni_correction]  #bonferroni correction
    #logger.info('\nBonferroni Correction: p<{}'.format(str(bonferroni_correction)))
    logger.info('Number of analyte pairs whose levels are significantly associated: {}'.format(str(df.shape[0])))
    df.to_csv(f_output_sig, index=False, sep='\t')
    
    # evaluate effect size - cohen's d
    logger.info('Effect Sizes (Cohen\'s D ) For Significant Correlations:')
    evaluate_effect_size(df, logger)
    logger.info('\nFinished all correlation analyses!')
    
    return df

In [20]:
os.remove('failed.spearman.standardized.interomics.' + CONDITION + '.pairs.all.miniDP3.txt')
os.remove('spearman.standardized.interomics.' + CONDITION + '.all.miniDP3.txt')
os.remove('spearman.standardized.interomics.' + CONDITION + '.sig.miniDP3.txt')

In [21]:
spearman_pairwise = run_spearman_pairwise(df, CONDITION, analytes, proteins, metabolites, miRNAs, transcripts, histopathology)

2024-02-04 20:56:44,792 - analytics - INFO - Created 63286875 pairs
2024-02-04 20:56:46,995 - analytics - INFO - Finished 1000 in 2.200 seconds (skipped 0)
2024-02-04 20:56:49,240 - analytics - INFO - Finished 2000 in 4.445 seconds (skipped 0)
2024-02-04 20:56:51,498 - analytics - INFO - Finished 3000 in 6.703 seconds (skipped 0)
2024-02-04 20:56:53,785 - analytics - INFO - Finished 4000 in 8.991 seconds (skipped 0)
2024-02-04 20:56:56,009 - analytics - INFO - Finished 5000 in 11.214 seconds (skipped 0)
2024-02-04 20:56:58,220 - analytics - INFO - Finished 6000 in 13.425 seconds (skipped 0)
2024-02-04 20:57:00,387 - analytics - INFO - Finished 7000 in 15.592 seconds (skipped 0)
2024-02-04 20:57:02,550 - analytics - INFO - Finished 8000 in 17.756 seconds (skipped 0)
2024-02-04 20:57:04,788 - analytics - INFO - Finished 9000 in 19.993 seconds (skipped 0)
2024-02-04 20:57:07,085 - analytics - INFO - Finished 10000 in 22.290 seconds (skipped 0)
2024-02-04 20:57:09,324 - analytics - INFO - 

2024-02-04 21:01:25,233 - analytics - INFO - Finished 91000 in 280.438 seconds (skipped 0)
2024-02-04 21:01:28,111 - analytics - INFO - Finished 92000 in 283.316 seconds (skipped 0)
2024-02-04 21:01:31,052 - analytics - INFO - Finished 93000 in 286.257 seconds (skipped 0)
2024-02-04 21:01:33,813 - analytics - INFO - Finished 94000 in 289.018 seconds (skipped 0)
2024-02-04 21:01:36,591 - analytics - INFO - Finished 95000 in 291.796 seconds (skipped 0)
2024-02-04 21:01:39,369 - analytics - INFO - Finished 96000 in 294.574 seconds (skipped 0)
2024-02-04 21:01:42,258 - analytics - INFO - Finished 97000 in 297.463 seconds (skipped 0)
2024-02-04 21:01:45,132 - analytics - INFO - Finished 98000 in 300.337 seconds (skipped 0)
2024-02-04 21:01:47,837 - analytics - INFO - Finished 99000 in 303.043 seconds (skipped 0)
2024-02-04 21:01:50,654 - analytics - INFO - Finished 100000 in 305.859 seconds (skipped 0)
2024-02-04 21:01:53,595 - analytics - INFO - Finished 101000 in 308.801 seconds (skipped 

2024-02-04 21:05:36,677 - analytics - INFO - Finished 181000 in 531.882 seconds (skipped 0)
2024-02-04 21:05:39,662 - analytics - INFO - Finished 182000 in 534.867 seconds (skipped 0)
2024-02-04 21:05:42,766 - analytics - INFO - Finished 183000 in 537.971 seconds (skipped 0)
2024-02-04 21:05:45,560 - analytics - INFO - Finished 184000 in 540.765 seconds (skipped 0)
2024-02-04 21:05:48,397 - analytics - INFO - Finished 185000 in 543.602 seconds (skipped 0)
2024-02-04 21:05:51,106 - analytics - INFO - Finished 186000 in 546.311 seconds (skipped 0)
2024-02-04 21:05:53,951 - analytics - INFO - Finished 187000 in 549.156 seconds (skipped 0)
2024-02-04 21:05:56,561 - analytics - INFO - Finished 188000 in 551.767 seconds (skipped 0)
2024-02-04 21:05:59,211 - analytics - INFO - Finished 189000 in 554.416 seconds (skipped 0)
2024-02-04 21:06:01,582 - analytics - INFO - Finished 190000 in 556.787 seconds (skipped 0)
2024-02-04 21:06:03,919 - analytics - INFO - Finished 191000 in 559.124 seconds 

2024-02-04 21:09:46,993 - analytics - INFO - Finished 271000 in 782.198 seconds (skipped 0)
2024-02-04 21:09:49,378 - analytics - INFO - Finished 272000 in 784.583 seconds (skipped 0)
2024-02-04 21:09:51,667 - analytics - INFO - Finished 273000 in 786.872 seconds (skipped 0)
2024-02-04 21:09:53,972 - analytics - INFO - Finished 274000 in 789.177 seconds (skipped 0)
2024-02-04 21:09:56,412 - analytics - INFO - Finished 275000 in 791.617 seconds (skipped 0)
2024-02-04 21:09:58,979 - analytics - INFO - Finished 276000 in 794.185 seconds (skipped 0)
2024-02-04 21:10:01,217 - analytics - INFO - Finished 277000 in 796.423 seconds (skipped 0)
2024-02-04 21:10:03,450 - analytics - INFO - Finished 278000 in 798.655 seconds (skipped 0)
2024-02-04 21:10:05,807 - analytics - INFO - Finished 279000 in 801.012 seconds (skipped 0)
2024-02-04 21:10:08,141 - analytics - INFO - Finished 280000 in 803.347 seconds (skipped 0)
2024-02-04 21:10:10,449 - analytics - INFO - Finished 281000 in 805.655 seconds 

2024-02-04 21:13:47,165 - analytics - INFO - Finished 360000 in 1022.370 seconds (skipped 0)
2024-02-04 21:13:49,682 - analytics - INFO - Finished 361000 in 1024.887 seconds (skipped 0)
2024-02-04 21:13:52,194 - analytics - INFO - Finished 362000 in 1027.399 seconds (skipped 0)
2024-02-04 21:13:54,721 - analytics - INFO - Finished 363000 in 1029.927 seconds (skipped 0)
2024-02-04 21:13:57,225 - analytics - INFO - Finished 364000 in 1032.431 seconds (skipped 0)
2024-02-04 21:13:59,800 - analytics - INFO - Finished 365000 in 1035.006 seconds (skipped 0)
2024-02-04 21:14:02,405 - analytics - INFO - Finished 366000 in 1037.610 seconds (skipped 0)
2024-02-04 21:14:04,971 - analytics - INFO - Finished 367000 in 1040.176 seconds (skipped 0)
2024-02-04 21:14:07,458 - analytics - INFO - Finished 368000 in 1042.663 seconds (skipped 0)
2024-02-04 21:14:09,962 - analytics - INFO - Finished 369000 in 1045.168 seconds (skipped 0)
2024-02-04 21:14:12,808 - analytics - INFO - Finished 370000 in 1048.0

...

2024-02-05 04:30:36,560 - analytics - INFO - Finished 10750000 in 27231.765 seconds (skipped 0)
2024-02-05 04:30:38,853 - analytics - INFO - Finished 10751000 in 27234.059 seconds (skipped 0)
2024-02-05 04:30:41,267 - analytics - INFO - Finished 10752000 in 27236.472 seconds (skipped 0)
2024-02-05 04:30:43,565 - analytics - INFO - Finished 10753000 in 27238.770 seconds (skipped 0)
2024-02-05 04:30:45,824 - analytics - INFO - Finished 10754000 in 27241.029 seconds (skipped 0)
2024-02-05 04:30:48,081 - analytics - INFO - Finished 10755000 in 27243.287 seconds (skipped 0)
2024-02-05 04:30:50,295 - analytics - INFO - Finished 10756000 in 27245.500 seconds (skipped 0)
2024-02-05 04:30:52,901 - analytics - INFO - Finished 10757000 in 27248.107 seconds (skipped 0)
2024-02-05 04:30:55,659 - analytics - INFO - Finished 10758000 in 27250.864 seconds (skipped 0)
2024-02-05 04:30:58,434 - analytics - INFO - Finished 10759000 in 27253.639 seconds (skipped 0)
2024-02-05 04:31:01,194 - analytics - IN

2024-02-05 04:34:14,038 - analytics - INFO - Finished 10836000 in 27449.243 seconds (skipped 0)
2024-02-05 04:34:16,492 - analytics - INFO - Finished 10837000 in 27451.697 seconds (skipped 0)
2024-02-05 04:34:18,860 - analytics - INFO - Finished 10838000 in 27454.065 seconds (skipped 0)
2024-02-05 04:34:21,132 - analytics - INFO - Finished 10839000 in 27456.337 seconds (skipped 0)
2024-02-05 04:34:23,438 - analytics - INFO - Finished 10840000 in 27458.644 seconds (skipped 0)
2024-02-05 04:34:25,724 - analytics - INFO - Finished 10841000 in 27460.930 seconds (skipped 0)
2024-02-05 04:34:27,993 - analytics - INFO - Finished 10842000 in 27463.199 seconds (skipped 0)
2024-02-05 04:34:30,299 - analytics - INFO - Finished 10843000 in 27465.505 seconds (skipped 0)
2024-02-05 04:34:32,561 - analytics - INFO - Finished 10844000 in 27467.766 seconds (skipped 0)
2024-02-05 04:34:34,848 - analytics - INFO - Finished 10845000 in 27470.053 seconds (skipped 0)
2024-02-05 04:34:37,110 - analytics - IN

2024-02-05 04:37:31,601 - analytics - INFO - Finished 10922000 in 27646.807 seconds (skipped 0)
2024-02-05 04:37:33,874 - analytics - INFO - Finished 10923000 in 27649.079 seconds (skipped 0)
2024-02-05 04:37:36,108 - analytics - INFO - Finished 10924000 in 27651.313 seconds (skipped 0)
2024-02-05 04:37:38,382 - analytics - INFO - Finished 10925000 in 27653.588 seconds (skipped 0)
2024-02-05 04:37:40,639 - analytics - INFO - Finished 10926000 in 27655.844 seconds (skipped 0)
2024-02-05 04:37:42,897 - analytics - INFO - Finished 10927000 in 27658.102 seconds (skipped 0)
2024-02-05 04:37:45,316 - analytics - INFO - Finished 10928000 in 27660.521 seconds (skipped 0)
2024-02-05 04:37:47,603 - analytics - INFO - Finished 10929000 in 27662.808 seconds (skipped 0)
2024-02-05 04:37:49,887 - analytics - INFO - Finished 10930000 in 27665.092 seconds (skipped 0)
2024-02-05 04:37:52,125 - analytics - INFO - Finished 10931000 in 27667.331 seconds (skipped 0)
2024-02-05 04:37:54,381 - analytics - IN

2024-02-05 04:40:50,496 - analytics - INFO - Finished 11008000 in 27845.701 seconds (skipped 0)
2024-02-05 04:40:53,314 - analytics - INFO - Finished 11009000 in 27848.520 seconds (skipped 0)
2024-02-05 04:40:56,105 - analytics - INFO - Finished 11010000 in 27851.310 seconds (skipped 0)
2024-02-05 04:40:58,866 - analytics - INFO - Finished 11011000 in 27854.072 seconds (skipped 0)
2024-02-05 04:41:01,650 - analytics - INFO - Finished 11012000 in 27856.855 seconds (skipped 0)
2024-02-05 04:41:04,427 - analytics - INFO - Finished 11013000 in 27859.632 seconds (skipped 0)
2024-02-05 04:41:07,187 - analytics - INFO - Finished 11014000 in 27862.393 seconds (skipped 0)
2024-02-05 04:41:09,640 - analytics - INFO - Finished 11015000 in 27864.845 seconds (skipped 0)
2024-02-05 04:41:12,245 - analytics - INFO - Finished 11016000 in 27867.450 seconds (skipped 0)
2024-02-05 04:41:14,584 - analytics - INFO - Finished 11017000 in 27869.789 seconds (skipped 0)
2024-02-05 04:41:16,857 - analytics - IN

2024-02-05 04:44:21,699 - analytics - INFO - Finished 11094000 in 28056.904 seconds (skipped 0)
2024-02-05 04:44:24,056 - analytics - INFO - Finished 11095000 in 28059.261 seconds (skipped 0)
2024-02-05 04:44:26,385 - analytics - INFO - Finished 11096000 in 28061.590 seconds (skipped 0)
2024-02-05 04:44:28,637 - analytics - INFO - Finished 11097000 in 28063.842 seconds (skipped 0)
2024-02-05 04:44:30,910 - analytics - INFO - Finished 11098000 in 28066.115 seconds (skipped 0)
2024-02-05 04:44:33,186 - analytics - INFO - Finished 11099000 in 28068.391 seconds (skipped 0)
2024-02-05 04:44:35,456 - analytics - INFO - Finished 11100000 in 28070.661 seconds (skipped 0)
2024-02-05 04:44:37,729 - analytics - INFO - Finished 11101000 in 28072.934 seconds (skipped 0)
2024-02-05 04:44:40,010 - analytics - INFO - Finished 11102000 in 28075.215 seconds (skipped 0)
2024-02-05 04:44:42,273 - analytics - INFO - Finished 11103000 in 28077.479 seconds (skipped 0)
2024-02-05 04:44:44,553 - analytics - IN

In [22]:
spearman_pairwise = pd.read_csv('spearman.standardized.interomics.' + CONDITION + '.sig.miniDP3.txt', sep='\t')

In [23]:
spearman_pairwise['coef'].hist(range=[-1, 1])

In [24]:
abs(spearman_pairwise['coef']).hist(range=[0, 1])

In [25]:
# filter for effect
df_no_effect = spearman_pairwise[abs(spearman_pairwise['coef'])<0.01]
df_very_small = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.01]
df_very_small = df_very_small[abs(df_very_small['coef'])<0.1]
df_effect = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.1]
print('\nNumber of analyte pairs whose effect size is at least weak: '.format(str(df_effect.shape[0])))


# evaluate effect size - cohen's d
df_small = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.1]
df_small = df_small[abs(df_small['coef'])<0.2]
df_medium = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.2]
df_medium  = df_medium[abs(df_medium['coef'])<0.3]
df_medium_large = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.3]
df_medium_large = df_medium_large[abs(df_medium_large['coef'])<0.5]
df_large = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.5]
df_large = df_large[abs(df_large['coef'])<0.7]
df_very_large = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.7]
df_very_large = df_very_large[abs(df_very_large['coef'])<0.9]
df_huge = spearman_pairwise[abs(spearman_pairwise['coef'])>=0.9]

# print info on effect sizes
print('Number of Analyte Pairs with No Monotonic Relationship: {}'.format(str(df_no_effect.shape[0])))
print('Number of Analyte Pairs with Very Weak Monotonic Relationship: {}'.format(str(df_very_small.shape[0])))
print('Number of Analyte Pairs with Weak Monotonic Relationship: {}'.format(str(df_small.shape[0])))
print('Number of Analyte Pairs with Moderate Monotonic Relationship: {}'.format(str(df_medium.shape[0])))
print('Number of Analyte Pairs with Moderate Strong Monotonic Relationship: {}'.format(str(df_medium_large.shape[0])))
print('Number of Analyte Pairs with Strong Monotonic Relationship: {}'.format(str(df_large.shape[0])))
print('Number of Analyte Pairs with Very Strong Monotonic Relationship: {}'.format(str(df_very_large.shape[0])))
print('Number of Analyte Pairs with Near Perfect Monotonic Relationship: {}'.format(str(df_huge.shape[0])))